# 01 - GEE Connection Test

Verify Google Earth Engine authentication and basic queries work.

In [ ]:
import ee
import sys
sys.path.insert(0, '..')

# GCP Project ID
GCP_PROJECT_ID = 'gen-lang-client-0669818939'

## 1. Initialize GEE

In [ ]:
# Authenticate (run once)
# ee.Authenticate()

# Initialize with project
ee.Initialize(project=GCP_PROJECT_ID)
print(f'GEE initialized with project: {GCP_PROJECT_ID}')

## 2. Test Dataset Access

In [ ]:
# Delhi region
DELHI_BOUNDS = [76.8, 28.4, 77.4, 28.9]  # [lng_min, lat_min, lng_max, lat_max]
delhi_region = ee.Geometry.Rectangle(DELHI_BOUNDS)

# Dataset IDs to test
DATASETS = {
    'AlphaEarth': 'GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL',
    'DEM': 'USGS/SRTMGL1_003',
    'Surface Water': 'JRC/GSW1_4/GlobalSurfaceWater',
    'Precipitation': 'UCSB-CHG/CHIRPS/DAILY',
    'ERA5 Land': 'ECMWF/ERA5_LAND/DAILY_AGGR',
    'Landcover': 'ESA/WorldCover/v200'
}

In [ ]:
# Test each dataset
for name, dataset_id in DATASETS.items():
    print(f'\nTesting {name}...')
    try:
        if 'DAILY' in dataset_id or 'ANNUAL' in dataset_id:
            collection = ee.ImageCollection(dataset_id)
            count = collection.filterBounds(delhi_region).limit(10).size().getInfo()
            print(f'  Collection size (sample): {count}')
        else:
            image = ee.Image(dataset_id)
            bands = image.bandNames().getInfo()
            print(f'  Bands: {bands[:5]}...' if len(bands) > 5 else f'  Bands: {bands}')
        print(f'  Status: OK')
    except Exception as e:
        print(f'  ERROR: {e}')

## 3. Test Basic Query

In [ ]:
# Get elevation statistics for Delhi
dem = ee.Image('USGS/SRTMGL1_003')

stats = dem.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.min(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.max(),
        sharedInputs=True
    ),
    geometry=delhi_region,
    scale=100,
    maxPixels=1e9
).getInfo()

print('Delhi Elevation Stats:')
print(f"  Mean: {stats.get('elevation_mean', 'N/A'):.1f}m")
print(f"  Min: {stats.get('elevation_min', 'N/A'):.1f}m")
print(f"  Max: {stats.get('elevation_max', 'N/A'):.1f}m")

## 4. Summary

If all tests pass, GEE is properly configured and we can proceed with data exploration.